In [ ]:
! pip install TTS wandb

In [ ]:
! tts --list_models

In [ ]:
!tts --text "Bonjour, j'aime bien la viande. Et je suis vraiment content de te voir !" --model_name "tts_models/multilingual/multi-dataset/xtts_v2" --speaker_wav "./reference_speaker.wav" --out_path "../output.wav" --language_idx fr 

In [ ]:
import IPython
IPython.display.Audio("../output.wav")

In [74]:
from datasets import load_dataset, DatasetDict, concatenate_datasets
import os
import re

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [65]:
dataset = load_dataset('sil-ai/bloom-speech', 'bm', use_auth_token=True)
dataset

Generating train split: 0 examples [00:00, ? examples/s]

https://s3.amazonaws.com/bloom-speech/audio/bam-0000-i0543c85c-fa96-404d-9faa-7d024b5b1f39.mp3 failed to download. Data may be missing.


Generating test split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['file', 'audio', 'text', 'book', 'instance', 'license', 'credits', 'original_lang_tag'],
        num_rows: 203
    })
    test: Dataset({
        features: ['file', 'audio', 'text', 'book', 'instance', 'license', 'credits', 'original_lang_tag'],
        num_rows: 50
    })
    validation: Dataset({
        features: ['file', 'audio', 'text', 'book', 'instance', 'license', 'credits', 'original_lang_tag'],
        num_rows: 50
    })
})

In [69]:
tts_ds = DatasetDict({"train": concatenate_datasets([dataset['train'], dataset['validation']])})
tts_ds

DatasetDict({
    train: Dataset({
        features: ['file', 'audio', 'text', 'book', 'instance', 'license', 'credits', 'original_lang_tag'],
        num_rows: 253
    })
})

In [1]:
tts_ds['train'][0]

NameError: name 'tts_ds' is not defined

In [87]:
IPython.display.Audio(tts_ds['train'][0]['file'])

In [92]:
def number_to_bambara(n):
    """
    Convert a number into its textual representation in Bambara using recursion.

    Args:
    n (int): The number to be converted.

    Returns:
    str: The number expressed in Bambara text.

    Examples:
    >>> number_to_bambara(123)
    'kɛmɛ ni mugan ni saba'

    Notes:
    This function assumes that 'n' is a non-negative integer.
    """

    # Bambara numbering rules
    units = ["", "kɛlɛn", "fila", "saba", "naani", "duuru", "wɔrɔ", "wòlonwula", "sɛɛgin", "kɔnɔntɔn"]
    tens = ["", "tan", "mugan", "bisaba", "binaani", "biduuru", "biwɔrɔ", "biwòlonfila", "bisɛɛgin", "bikɔnɔntɔn"]
    hundreds = ["", "kɛmɛ"]
    thousands = ["", "waga"]
    millions = ["", "milyɔn"]
    
    # Handle zero explicitly
    if n == 0:
        return ""

    if n < 10:
        return units[n]
    elif n < 100:
        return tens[n // 10] if n < 20 else tens[n // 10] + (" ni " + number_to_bambara(n % 10) if n % 10 > 0 else "")
    elif n < 1000:
        return hundreds[1] + (" " + number_to_bambara(n // 100) if n >= 200 else "") + (" ni " + number_to_bambara(n % 100) if n % 100 > 0 else "")
    elif n < 1_000_000:
        return thousands[1] + " " + number_to_bambara(n // 1000) + (" ni " + number_to_bambara(n % 1000) if n % 1000 > 0 else "")
    else:
        return millions[1] + " " + number_to_bambara(n // 1_000_000) + (" ni " + number_to_bambara(n % 1_000_000) if n % 1_000_000 > 0 else "")
    
def normalize_bambara(text):
    """
    Normalize Bambara text for TTS by replacing numerical figures with their word equivalents.

    Args:
    text (str): The text to be normalized.

    Returns:
    str: The normalized Bambara text.
    """

    # A regex pattern to match all numbers
    number_pattern = re.compile(r'\b\d+\b')

    # Function to replace each number with its Bambara text
    def replace_number_with_text(match):
        number = int(match.group())
        return number_to_bambara(number)

    # Replace each number in the text with its Bambara word equivalent
    normalized_text = number_pattern.sub(replace_number_with_text, text)

    return normalized_text

# Function to create the metadata file
def create_metadata_file(dataset, output_dir='MyTTSDataSet'):
    # Create the output directory if it doesn't exist
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    
    # Define the path to the metadata file
    metadata_path = os.path.join(output_dir, 'metadata.txt')

    # Open the metadata file in write mode
    with open(metadata_path, 'w', encoding='utf-8') as f:
        # Iterate over each item in the dataset
        for item in dataset['train']:
            # Your dataset should have an 'audio' column with a dictionary containing the file path and 'array' for the audio data
            audio_path = item['audio']['path']
            bambara_text = item['text'].replace(" ", " ").replace(" ", " ").replace("\n", " ")
            normalized_text = normalize_bambara(bambara_text)

            # Write the formatted data to the metadata file
            f.write(f"{audio_path}|{bambara_text}|{normalized_text}\n")
    
    return metadata_path

In [93]:
create_metadata_file(tts_ds, output_dir="./dataset")

'../my_test_tts_data/metadata.txt'

In [94]:
! CUDA_VISIBLE_DEVICES="0" python ./train_gpt_xtts.py

>> DVAE weights restored from: /home/aboubacar/lab/others/bambara-translation/code/coqui-TTS/bam_recipes/run/training/XTTS_v2.0_original_model_files/dvae.pth
 | > Found 253 files in /home/aboubacar/lab/others/bambara-translation/code/coqui-TTS/my_test_tts_data
 > Training Environment:
 | > Backend: Torch
 | > Mixed precision: False
 | > Precision: float32
 | > Num. of CPUs: 20
 | > Num. of Torch Threads: 1
 | > Torch seed: 1
 | > Torch CUDNN: True
 | > Torch CUDNN deterministic: False
 | > Torch CUDNN benchmark: False
 | > Torch TF32 MatMul: False
Traceback (most recent call last):
  File "/home/aboubacar/lab/others/bambara-translation/code/coqui-TTS/notebooks/../bam_recipes/train_gpt_xtts.py", line 176, in <module>
    main()
  File "/home/aboubacar/lab/others/bambara-translation/code/coqui-TTS/notebooks/../bam_recipes/train_gpt_xtts.py", line 159, in main
    trainer = Trainer(
  File "/home/aboubacar/miniconda3/lib/python3.10/site-packages/trainer/trainer.py", line 440, in __init__
